**Roteiro - app**


**Primeira etapa (Espiral)**:


*   usuário possivelmente preenche algum form, com nome, idade, sexo.
*   em seguida é levado para a primeira página do exame: uma imagem template de espiral no background, onde ele deve tentar contorná-la com o dedo, e só soltar quando terminar.
*   Enquanto ele desenha, capturamos o máximos de informações relativas ao tremor do celular (ex: aceleração em x, y, z, ...).

**Segunda etapa (Meandro)**:

*   Terminado o desenho da espiral, levá-lo a uma próxima página, com um meandro de background agora. Ele deve fazer a mesma tarefa, sendo que vamos capturar as mesmas informações.

**Terceira etapa (Diadochokinese)**:

*   Terminado o componente do meandro, o usuário vai para o teste de diadochokinese (basicamente, ele deve estender o braço segurando o celular por um tempo determinado), e capturaremos também o tremor dele.

**Quarta etapa (Voz)**:

*   Terminado a etapa de diadochokinese, ele vai a uma próxima página com uma frase indicando para ele falar um fonema pre-determinado, e um botão para ele apertar, falar e soltar quando terminar.
*   No final de tudo, enviar todas as informações pro nosso backend via **POST** (**imagem do desenho da espiral, imagem do desenho do meandro, 2 sinais temporais do tremor, cada um correspondente a um desenho, e um sinal de voz**)

**O que já foi feito**:

*   [ X ] Componente inicial com input de idade e botão de começar.
*   [ X ] Componente da espiral, capturando as informações de tremor quando o user começa o desenho.
*   [ X ] Componente do meandro, capturando as informações de tremor quando o user começa o desenho.
*   [ X ] Componente de diadochokinese.
*   [ X ] Componente de áudio, há um botão simples que grava o áudio.
*   [ X ] Adicionar animações (success, load)
*   [ X ] Colocar uma animação na tela para mostrar que o teste está em andamento, e um alert quando o teste tiver acabado (diadochokinese).

**TO-DO**:

*   [  ] **O mais urgente**: no componente de diadochokinese, nosso timer está bugando (porque o React Native renderiza varias vezes por segundo, consequentemente criando as variáveis que declaramos várias vezes. A solução disso está possivelmente relacionada com o uso do useEffect). O que acontece é que vários timers acabam sendo criados por segundo, então o timer acaba cumulando os tempos (ex: se era para acabar em 3s, ele acumula e fica 7s).
*   [  ] **Segundo mais urgente**: a biblioteca para pegar informações dos sensores é a 'react-native-sensors'. Para começar a receber informações dos sensores, deve-se utilizar o método de "subscribe", que é como um listener que fica escutando os sensores do celular. Nosso outro problema está sendo em fazer o "unsubscribe", ou seja, parar de escutar, remover nosso listener. Está sendo difícil também pelo problema das várias renderizações do react native.
*   [  ] Colocar um logo na tela inicial.
*   [  ] Estilizar página de áudio (colocar botão estilizado de gravação).
*   [  ] Colocar algum aviso na hora do teste de diadochokinese para explicar como o user deve segurar o celular
*   [  ] Testar permissão para audio no caso do iPhone.
*   [  ] Enviar via POST todos os dados para a rota do backend.

**OBS**: se na hora de rodar "npx react-native run-android" ele der um erro devido à biblioteca do react-native-sensors, basta atualizar o "min SDK" dentro do node_modules/react-native-sensors/build.gradle.


**##########################################################**

**Roteiro servidor**


**Espiral e Meandro**:
*   Recebendo via POST todas as informações acima, vamos rodar o algoritmo de extração das informações do artigo do Clayton, em C (deem uma olhada http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/source_code.tar.gz). A partir da imagem da espiral, por exemplo, esse algoritmo vai extrair informações como diferença máxima entre o desenho e o template da espiral, desvio padrão da diferença entre template e desenho, etc.
*   Depois de rodar o algoritmo para as imagens da espiral e do meandro, o arquivo .C deve gerar 2 arquivos **txt** com essas informações que falei acima, um para cada desenho.
*   Teremos um modelo para classificar apenas informações da espiral, e um apenas para o meandro. Como a etapa de treinamento de nossos modelos já estarão feitas, apenas carregaremos do disco o arquivo de cada modelo.

**Sinais temporais**:


*   Vamos seguir a mesma linha de raciocínio do segundo artigo, onde o sinal temporal gerado pela caneta especial era basicamente uma matriz de **t** linhas por **6** colunas: **t** sendo o tempo que durou o desenho daquele paciente, e cada coluna representa um sinal de fato em algum instante **t** (então por exemplo, a terceira linha contém 6 valores, sendo que cada valor representa um sinal da caneta no terceiro instante, por exemplo, a primeira coluna, a pressão relativa, a segunda, o tremor em X, a terceira, o tremor em Y etc..). Em outras palavras, teremos sendo enviado via POST 2 matrizes, uma representando cada sinal.
*   A partir de cada matriz, vamos aplicar algumas operações para transformá-la nos níveis de cinza, pois vamos transformá-la em uma imagem, essa foi a ideia do segundo artigo (se quiserem dar uma olhada sem se preocupar com os detalhes, http://sibgrapi.sid.inpe.br/col/sid.inpe.br/sibgrapi/2016/07.08.22.47/doc/opf-sibgrapi16.pdf).
*   A grande sacada do artigo foi pegar a imagem e treinar uma rede neural para realizar a classificação! No caso vamos testar algumas arquiteturas já bem conhecidas para reconhecimento de imagens, como imageNet, Inception, CIFAR etc..
*   **Não** utilizaremos aprendizado por transferência, apenas aproveitaremos a arquitetura de cada rede.
*   Pensei em utilizar alguma biblioteca como pyTorch por ser rápida nos benchmarks. Porém isso temos que discutir também.


**Sinal de voz**:

*   A partir do sinal de voz recebido, realizarmos alguns pré-processamentos no áudio para então extrair várias características como Jitter, Shimmer, f0, Harmonic-to-noise ratio etc..
*   Utilizaremos transformadas wavelets.

**endpoint**:

*   Nosso app vai chamar uma rota do nosso backend que será um python escutando pelo flask.

**Plataforma**:

*   AWS ou Google Cloud.


**##########################################################**

**O que já foi feito (servidor)**:
*   código em flask bem simples ainda, lembrando que não vamos usar Flask no deploy (https://github.com/jogisuda/parkinson/blob/master/server.ipynb).
*   Parte de áudio, já tem algumas coisas montadas, como nossa extração de características via biblioteca do **parselmouth** (https://github.com/jogisuda/parkinson/blob/master/audio.ipynb)

**Quando recebermos via POST a imagem do desenho da Espiral:**

In [ ]:
#carregar nosso modelo treinado

#receber imagem da Espiral (via POST por ex)

#executar extração de carac. (algoritmo do artigo em c)

#ler .txt em numpyArray para armazenar features (IMPORTANTE: checar se as features podem ser negativas, pois deve ser tratado no numpy.loadtxt)

#normalizar as características

#espiralResult = SVMSpiral.predict()

**Para desenho do Meandro**

In [ ]:
#carregar nosso modelo treinado

#receber imagem do meandro (via POST por ex)

#executar extração de carac. (algoritmo do artigo em c)

#ler .txt em numpyArray para armazenar features

#normalizar as características

#espiralResult = modeloMeandro.predict()


**Para o sinal dos tremores (ex: aceleração em x, y, z, ...) - o objetivo será seguir a mesma linha de raciocínio do último artigo do Papa**:

*   transformaremos nossas informações do tremor em uma **imagem** de tamanho  padronizado: para isso, a matriz de valores deve ser transformada em uma matr. **quadrada**, e depois normalizada para os níveis de **cinza**
*   Com nossa imagem já preparada, jogaremos na CNN para o predict()

*   Testaremos diferentes arquiteturas para nossa rede, por ex: imageNet, Inception, CIFAR etc.
*   **Não** utilizaremos transfer learning, já que essas redes não são treinadas no contexto do nosso foco.



In [ ]:
#carregar nosso modelo treinado

#receber matriz de sinal do tremor via POST (r linhas, n colunas: r sendo cada tempo de amostragem, n sendo cada canal (ex: tremor X, tremor Y, ...))

#deixar a matrz quadrada

#normalizar pros niveis de cinza

#jogar na CNN (resNet, possibilidades são: caffe/pyTorch, TensorFlow de bibliotecas)